<h1><center><img align="center" src="Pix/logostodos.png" height="100" style="height:100px;"></center></h1>

In [1]:
%reset -f
%store -z 

In [2]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm, tqdm_notebook, tnrange
import time
import pickle

import datetime
from datetime import date

from IPython.display import clear_output, display
from ipywidgets import HTML

from math import sin, cos, sqrt, atan2, radians

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import shap

In [3]:
model_gravedad = pickle.load(open('Models/best_model_severity.pkl', 'rb'))
#X_shap=pd.read_pickle('datos/Prediccion/datos_para_shap.pkl')

modelos=[model_gravedad]

<h1><center>Herramienta de predicción de la gravedad de los pacientes</center></h1>

## Data introduction:

In [5]:
style = {'description_width': 'initial'}
age=widgets.Text(
    value='',
    placeholder='71',
    description='Age:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='30%')
)
last_o2_sat=widgets.Text(
    value='',
    placeholder='95.0000',
    description='Oxygen saturation:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='30%')
)

P70_chronic=widgets.Dropdown(
    options=[('No',0),('Yes',1)],
    value=0,
    description='Diagnosed dementia:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
T82_chronic=widgets.Dropdown(
    options=[('No',0),('Yes',1)],
    value=0,
    description='Obesity:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
K89_chronic=widgets.Dropdown(
    options=[('No',0),('Yes',1)],
    value=0,
    description='Cerebrovascular disease:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
K92_chronic=widgets.Dropdown(
    options=[('No',0),('Yes',1)],
    value=0,
    description='Intermittent claudication:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)

widgets.HBox([age, last_o2_sat, widgets.Label(value="%")], layout=widgets.Layout(width='100%'))

### Diagnostics:

In [8]:
widgets.HBox([P70_chronic, T82_chronic], layout=widgets.Layout(width='100%'))

In [9]:
widgets.HBox([K89_chronic, K92_chronic], layout=widgets.Layout(width='100%'))

### Lab tests:

In [13]:
UREA=widgets.Text(
    value='',
    placeholder='0.4185',
    description='UREA:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
NE=widgets.Text(
    value='',
    placeholder='74.9150',
    description='NEUTROPHILS %:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
EOR=widgets.Text(
    value='',
    placeholder='0.0065',
    description='EOSINOPHILS :', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
LY=widgets.Text(
    value='',
    placeholder='15.7000',
    description='LYMPHOCYTES %:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
LDH=widgets.Text(
    value='',
    placeholder='322.5000',
    description='LDH:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
GLUC=widgets.Text(
    value='',
    placeholder='120.0000',
    description='BASAL GLUCOSE:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
INR=widgets.Text(
    value='',
    placeholder='1.1150',
    description='INR-TP:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
CREA=widgets.Text(
    value='',
    placeholder='0.9100',
    description='CREATININE:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
EO=widgets.Text(
    value='',
    placeholder='0.1000',
    description='EOSINOPHILS %:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
RBC=widgets.Text(
    value='',
    placeholder='4.5300',
    description='RED BLOOD CELLS:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
CL=widgets.Text(
    value='',
    placeholder='101.8761',
    description='CHLORIDE', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
LYR=widgets.Text(
    value='',
    placeholder='0.9759',
    description='LYMPHOCYTES:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
MO=widgets.Text(
    value='',
    placeholder='7.6000',
    description='MONOCYTES %:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)
VCM=widgets.Text(
    value='',
    placeholder='90.0000',
    description='MCV:', style={'description_width': 'initial'},
    disabled=False, layout=widgets.Layout(width='45%')
)

In [14]:
widgets.HBox([RBC, widgets.Label(value=" mill/mm3", layout=widgets.Layout(width='5%')),
             VCM, widgets.Label(value=" fl", layout=widgets.Layout(width='5%'))], layout=widgets.Layout(width='100%'))

In [15]:
widgets.HBox([NE, widgets.Label(value="%", layout=widgets.Layout(width='5%')),
             LY, widgets.Label(value=" %", layout=widgets.Layout(width='5%'))], layout=widgets.Layout(width='100%'))

In [16]:
widgets.HBox([MO, widgets.Label(value=" %", layout=widgets.Layout(width='5%')),
             EO, widgets.Label(value=" %", layout=widgets.Layout(width='5%'))], layout=widgets.Layout(width='100%'))

In [17]:
widgets.HBox([LYR, widgets.Label(value=" mil/mm3", layout=widgets.Layout(width='5%')),
             EOR, widgets.Label(value=" mil/mm3", layout=widgets.Layout(width='5%'))], layout=widgets.Layout(width='100%'))

In [18]:
widgets.HBox([INR, widgets.Label(value=" ", layout=widgets.Layout(width='5%')),
             GLUC, widgets.Label(value=" mg/dl", layout=widgets.Layout(width='5%'))], layout=widgets.Layout(width='100%'))

In [19]:
widgets.HBox([UREA, widgets.Label(value="g/L", layout=widgets.Layout(width='5%')), 
              CREA, widgets.Label(value="mg/dl", layout=widgets.Layout(width='5%'))], layout=widgets.Layout(width='100%'))

In [21]:
widgets.HBox([CL, widgets.Label(value=" mmol/l", layout=widgets.Layout(width='5%')), 
              LDH, widgets.Label(value=" U/l", layout=widgets.Layout(width='5%'))], layout=widgets.Layout(width='100%'))

## Results:

In [25]:
out = widgets.Output()

@out.capture()

def predecir(boton_pred, models_pred,UREA,NE,EOR,LY,LDH,GLUC,INR,CREA,EO,RBC,CL,LYR,
              MO,VCM,age,last_o2_sat,P70_chronic,T82_chronic,K89_chronic,K92_chronic):#, con):
    out.clear_output()

    dic_medianas={'edad': 71.0,
                 'UREA.lab': 0.4185,
                 'NE..lab': 74.915,
                 'EOR.lab': 0.0065,
                 'P70_cronico.diagnostico': 0.0,
                 'LY..lab': 15.7,
                 'LDH.lab': 322.5,
                 'GLUC.lab': 120.0,
                 'INR.lab': 1.1150000000000002,
                 'CREA.lab': 0.91,
                 'T82_cronico.diagnostico': 0.0,
                 'sat_o2_ultimo.constante': 95.0,
                 'EO..lab.1': 0.1,
                 'RBC.lab': 4.53,
                 'CL.lab': 101.87615288741195,
                 'LYR.lab': 0.9759,
                 'K89_cronico.diagnostico': 0.0,
                 'K92_cronico.diagnostico': 0.0,
                 'MO..lab': 7.6,
                 'VCM.lab': 90.0,
                 'salida': 0.0}
    with tqdm(total=2) as pbar:
        pbar.set_description("Recovering data:")
        
        variables=[age,UREA,NE,EOR,LY,LDH,GLUC,INR,CREA,last_o2_sat,EO,RBC,CL,LYR,
                   MO,VCM,T82_chronic,K89_chronic,K92_chronic,P70_chronic]
        variables=[float(var.value) if var.value!='' else np.nan for var in variables]
        order_cols=['edad', 'UREA.lab', 'NE..lab', 'EOR.lab', 'P70_cronico.diagnostico', 'LY..lab', 'LDH.lab', 'GLUC.lab', 
                'INR.lab', 'CREA.lab', 'T82_cronico.diagnostico', 'sat_o2_ultimo.constante', 'EO..lab.1', 'RBC.lab', 'CL.lab', 
                'LYR.lab', 'K89_cronico.diagnostico', 'K92_cronico.diagnostico', 'MO..lab', 'VCM.lab']
        order_cols2=['edad', 'UREA.lab', 'NE..lab', 'EOR.lab', 'LY..lab', 'LDH.lab', 
                      'GLUC.lab', 'INR.lab', 'CREA.lab', 'sat_o2_ultimo.constante', 'EO..lab.1', 'RBC.lab', 
                      'CL.lab', 'LYR.lab', 'MO..lab', 'VCM.lab', 'T82_cronico.diagnostico', 'K89_cronico.diagnostico', 
                      'K92_cronico.diagnostico', 'P70_cronico.diagnostico']
        data_pred=pd.DataFrame(np.asarray(variables).reshape((1,20)),
                     columns=order_cols2).fillna(value=dic_medianas)[order_cols]
        
#         data_shap=pd.concat([data_pred,X_shap])[order_cols]
#         explainer = shap.TreeExplainer(models_pred[0])
#         shap_values = explainer.shap_values(data_shap)
        
        pbar.update(1)
        pbar.set_description("Obtaining severity prediction:")
        pred_grav=models_pred[0].predict_proba(data_pred)[0][1]
        pred_grav_str='No'
        if pred_grav>0.4:
            pred_grav_str='Yes'
        pred_text.value=pred_grav_str
        pred_prob_text.value=str(pred_grav)

        pbar.update(1)
        pbar.set_description("Done!")
        
#         shap.force_plot(explainer.expected_value, shap_values[0,:], data_shap.iloc[0,:], matplotlib=True)

boton=widgets.ToggleButton(
    description='Predict',
    disabled=False,
    button_style='',
    tooltip='Click here to obtain the predictions',
    icon='check',
    layout=widgets.Layout(width='20%')
)

widgets.VBox([out, boton], layout=widgets.Layout(width='100%'))

In [26]:
pred_prob_text=widgets.Text(
    value='',
    placeholder='',
    description='Probability of becoming severe: ', style={'description_width': 'initial'},
    disabled=True, layout=widgets.Layout(width='45%')
)
pred_text=widgets.Text(
    value='',
    placeholder='',
    description='Will become severe: ', style={'description_width': 'initial'},
    disabled=True, layout=widgets.Layout(width='45%')
)


models_interactive = widgets.interactive(predecir, boton_pred=boton, models_pred=widgets.fixed(modelos),
#                                           X_shap=widgets.fixed(X_shap),
                                          UREA=widgets.fixed(UREA),NE=widgets.fixed(NE),EOR=widgets.fixed(EOR),
                                          LY=widgets.fixed(LY),LDH=widgets.fixed(LDH),GLUC=widgets.fixed(GLUC),
                                          INR=widgets.fixed(INR),CREA=widgets.fixed(CREA),EO=widgets.fixed(EO),
                                          RBC=widgets.fixed(RBC),CL=widgets.fixed(CL),LYR=widgets.fixed(LYR),
                                          MO=widgets.fixed(MO),VCM=widgets.fixed(VCM),age=widgets.fixed(age),
                                          last_o2_sat=widgets.fixed(last_o2_sat),P70_chronic=widgets.fixed(P70_chronic),
                                          T82_chronic=widgets.fixed(T82_chronic),K89_chronic=widgets.fixed(K89_chronic),
                                          K92_chronic=widgets.fixed(K92_chronic))

In [27]:
widgets.HBox([pred_prob_text, pred_text], layout=widgets.Layout(width='100%'))

<h1><center><img align="center" src="Pix/logoFEDER3.png" height="100" style="height:100px;"></center></h1>
